In [1]:
!pip install flair -U
!pip install allennlp==0.9.0 --no-deps
!pip install spacy
!pip uninstall lxml -y
!pip install lxml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 106.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 79.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 92.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 131.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 21.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  

In [1]:
import os
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, ELMoEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

In [2]:
def segment_data(data_file):
    try:
        import pandas as pd
    except ImportError:
        raise
    
    data = pd.read_csv(data_file, encoding='latin-1').sample(frac=1).drop_duplicates()
    data = data[['classes', 'title']].rename(columns={"classes":"label", "title":"text"})
    data['label'] = '__label__' +data['label'].astype(str)
    data['text'] = data['text'].apply(lambda k: k.lower().strip())
    data.to_csv('./data/whole.csv', sep='\t', index = False, header = False)
    data.iloc[0:int(len(data)*0.8)].to_csv('./data/train.csv', sep='\t', index = False, header = False)
    data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('./data/test.csv', sep='\t', index = False, header = False)
    data.iloc[int(len(data)*0.9):].to_csv('./data/dev.csv', sep='\t', index = False, header = False)
    return

In [3]:
def initialize_embeddings():
    """
    Summary:
        Stacks the list of pre-trained embedding vectors to be used as word representation (in concat.)
    Return:
        list: Returns list of pretrained embeddings vectors
    """
    word_embeddings = [
			WordEmbeddings('glove'),
			FlairEmbeddings('news-forward'),
			FlairEmbeddings('news-backward')
		]
    return word_embeddings
 
word_embeddings = initialize_embeddings()

In [ ]:
import flair.datasets
corpus = flair.datasets.UD_ENGLISH()
ll = corpus.make_label_dictionary(label_type = 'upos')
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256, rnn_type='LSTM', rnn_layers=1, bidirectional=False)

classifier = TextClassifier(document_embeddings, label_dictionary=ll, multi_label=False, label_type = 'upos')

trainer = ModelTrainer(classifier, corpus)

trainer.train('./model', max_epochs=20, patience=5, mini_batch_size=32, learning_rate=0.1)

2022-10-06 01:34:52,115 https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu not found in cache, downloading to /tmp/tmplavk_8ha


1738438B [00:00, 108837039.44B/s]        

2022-10-06 01:34:52,149 copying /tmp/tmplavk_8ha to cache at /root/.flair/datasets/ud_english/en_ewt-ud-dev.conllu
2022-10-06 01:34:52,151 removing temp file /tmp/tmplavk_8ha
2022-10-06 01:34:52,248 https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu not found in cache, downloading to /tmp/tmpzkxkpi_b



1738935B [00:00, 74259525.00B/s]         

2022-10-06 01:34:52,303 copying /tmp/tmpzkxkpi_b to cache at /root/.flair/datasets/ud_english/en_ewt-ud-test.conllu
2022-10-06 01:34:52,305 removing temp file /tmp/tmpzkxkpi_b


...

2022-10-06 01:37:17,470 Evaluating as a multi-label problem: False
2022-10-06 01:37:17,581 DEV : loss 0.020406417548656464 - f1-score (micro avg)  0.0
2022-10-06 01:37:17,707 BAD EPOCHS (no improvement): 0
2022-10-06 01:37:17,709 ----------------------------------------------------------------------------------------------------
2022-10-06 01:37:18,963 epoch 8 - iter 19/196 - loss 0.02169285 - samples/sec: 971.51 - lr: 0.100000
2022-10-06 01:37:20,209 epoch 8 - iter 38/196 - loss 0.02178150 - samples/sec: 978.50 - lr: 0.100000
2022-10-06 01:37:23,385 epoch 8 - iter 57/196 - loss 0.02229920 - samples/sec: 383.07 - lr: 0.100000
2022-10-06 01:37:24,707 epoch 8 - iter 76/196 - loss 0.02255851 - samples/sec: 921.44 - lr: 0.100000
2022-10-06 01:37:26,002 epoch 8 - iter 95/196 - loss 0.02236575 - samples/sec: 940.90 - lr: 0.100000
2022-10-06 01:37:27,257 epoch 8 - iter 114/196 - loss 0.02221352 - samples/sec: 970.46 - lr: 0.100000
2022-10-06 01:37:28,533 epoch 8 - iter 133/196 - loss 0.022125

100%|██████████| 32/32 [00:01<00:00, 19.10it/s]


2022-10-06 01:37:34,837 Evaluating as a multi-label problem: False
2022-10-06 01:37:34,948 DEV : loss 0.01813221164047718 - f1-score (micro avg)  0.0
2022-10-06 01:37:35,067 BAD EPOCHS (no improvement): 0
2022-10-06 01:37:35,070 ----------------------------------------------------------------------------------------------------
2022-10-06 01:37:36,421 epoch 9 - iter 19/196 - loss 0.01861413 - samples/sec: 902.19 - lr: 0.100000
2022-10-06 01:37:37,732 epoch 9 - iter 38/196 - loss 0.01965456 - samples/sec: 928.81 - lr: 0.100000
2022-10-06 01:37:39,060 epoch 9 - iter 57/196 - loss 0.01955340 - samples/sec: 917.44 - lr: 0.100000
2022-10-06 01:37:40,313 epoch 9 - iter 76/196 - loss 0.01944323 - samples/sec: 972.80 - lr: 0.100000
2022-10-06 01:37:41,653 epoch 9 - iter 95/196 - loss 0.01903195 - samples/sec: 909.14 - lr: 0.100000
2022-10-06 01:37:42,929 epoch 9 - iter 114/196 - loss 0.01893997 - samples/sec: 954.62 - lr: 0.100000
2022-10-06 01:37:44,255 epoch 9 - iter 133/196 - loss 0.0188187

100%|██████████| 32/32 [00:01<00:00, 19.22it/s]


2022-10-06 01:37:50,525 Evaluating as a multi-label problem: False
2022-10-06 01:37:50,641 DEV : loss 0.014614132232964039 - f1-score (micro avg)  0.0
2022-10-06 01:37:50,768 BAD EPOCHS (no improvement): 0
2022-10-06 01:37:50,872 ----------------------------------------------------------------------------------------------------
2022-10-06 01:37:52,192 epoch 10 - iter 19/196 - loss 0.01594162 - samples/sec: 923.45 - lr: 0.100000


In [4]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('./model/final-model.pt')

mapping = {
    'sad': '&#x1F61E',
    'smile': '&#x1F600',
    'food': '&#x1F37D',
    'heart': '&#10084;',
    'baseball': '&#x26be;'
}


def emoji(text):
        data = text
        if not len(data.strip()):
            return ''
        sentence = Sentence(data)
        classifier.predict(sentence)
        print (str(sentence.labels))
        if 'sad'in str(sentence.labels):
            return mapping['sad'] 
        elif 'smile' in str(sentence.labels):
            return mapping['smile'] 
        elif 'food' in str(sentence.labels):
            return mapping['food']
        elif 'heart' in str(sentence.labels):
            print (1)
            return mapping['heart']
        elif 'baseball' in str(sentence.labels):
            return mapping['baseball']

2022-10-06 01:38:14,249 loading file ./model/final-model.pt


In [ ]:
print(emoji('i ran home so quickly and smiled the whole way!'))
print(emoji('the sad little boy walked home alone in the heavy rain.'))


['Sentence: "i ran home so quickly and smiled the whole way !"'/'PRON' (0.9784)]
&#x1F600
['Sentence: "the sad little boy walked home alone in the heavy rain ."'/'DET' (0.635)]
&#x1F61E
